In [5]:
#搞不清楚读文件和列表的区别
#https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .config("sparksql.warehouse.dir","file:///c:/temp")\
    .appName("rdd_operation").getOrCreate()
spark.conf.set("spark.executor.memory","4g")
sc = spark.sparkContext#注意是sparkContext，不是SparkContext

#### 创建RDDS

In [6]:
data = sc.parallelize([("Amber",22),("Ember",23),("Pmber",24),("Gmber",12),
                       ("Amber",9)])

In [7]:
data.collect()#一般不要轻易使用，一般数据量大，使用take()

[('Amber', 22), ('Ember', 23), ('Pmber', 24), ('Gmber', 12), ('Amber', 9)]

In [8]:
data_from_file =sc.textFile('sales.csv',4)

#### Schema
rdds是schema-less的数据结构

In [10]:
data_hetergenous =sc.parallelize([("ferrari","fast"),{"porsche":100000},["spain","visited",4504]]).collect()
data_hetergenous
#可以是任何类型

[('ferrari', 'fast'), {'porsche': 100000}, ['spain', 'visited', 4504]]

In [11]:
#像python一样进行数据访问
data_hetergenous[1]['porsche']

100000

#### 从外部读入数据

In [12]:
data_from_file.take(1)

['1,34,2334']

user definded functions
复杂结构需要定义UDF函数

In [ ]:
def extractInformation(row):
    import re
    import numpy as np
    
    selected_indices = [ 2,3,4,5,6,7,8,9,10,11,12,13,14,
                       15,16,17,18,19,20,21,22,23,24,25,26,
                       27,28,29,30,31,32,33
                       ]    
   

In [ ]:
data_from_file_conv =data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row:row).take(5)

##transformations
##.map(...)

In [ ]:
data_2014 =data_from_file_conv.map(lambda row:int(row[16]))
#应用于每一个item上，list的第16项返回
data_2014.take(10)

In [ ]:
data_2014_2 =data_from_file_conv.map(lambda row:(row[16],int(row[16])))
#创建一个键值对，k-value
data_2014_2.take(10)

In [ ]:
data_2014_2.groupByKey().take(3)

.filter(...)#满足某个条件

In [15]:
acTransList =sc.parallelize([("SB10001",1000),("SB10002",1200),("SB10003",8000),("SB10004",4000),("SB10005",300),("SB10006",10000),("SB10007",500),("SB10008",56),("SB10009",1000),("B10009",1000),("SB10011",-1000)])
#acTransList.collect()
#data_filtered = acTransList.filter(lambda x: x[1]).filter(lambda x: x[0].startswith("SB"))
data_filtered = acTransList.filter(lambda x: x[1] and x[0].startswith("SB"))
data_filtered.count()

10

.flatMap(...)

In [21]:
data_flat = acTransList.flatMap(lambda x:(x[1],int(x[1])+1))
data_flat.take(4)
#这里为什么take(5)会报错呢？collect()出来有很多项

[1000, 1001, 1200, 1201]

.distinct()

In [22]:
distinct_gender = acTransList.map(lambda x:x[1]).distinct().collect()
distinct_gender

[1000, 1200, 8000, 4000, 300, 10000, 500, 56, -1000]

.sample(...)

In [29]:
fraction =0.9
data_sample = acTransList.sample(False,fraction,666)#第一个参数是有放回还是无放回，第二个参数是随机比例，第三个参数是i随机种子
data_sample.take(1)

[('SB10002', 1200)]

In [31]:
print("Original dataset:{0},sample:{1}".format(acTransList.count(),data_sample.count()))

Original dataset:11,sample:8


.leftOuterJoin(...)

In [40]:
rdd1 =sc.parallelize([("a",1),("b",1),("c",10)])
rdd2 =sc.parallelize([("a",1),("b",5),("c",2),("d",7)])
rdd3 =rdd1.leftOuterJoin(rdd2)
rdd3.collect()
#为什么take(2)不可以


[('a', (1, 1)), ('b', (1, 5)), ('c', (10, 2))]

In [ ]:
.join(...)

In [41]:
rdd4 =rdd1.join(rdd2)
rdd4.collect()

[('a', (1, 1)), ('b', (1, 5)), ('c', (10, 2))]

.intersection(...)

In [42]:
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

.repartition(...)

In [46]:
rdd1 =rdd1.repartition(4)
len(rdd1.glom().collect())

4

Actions

.take(...)

In [48]:
data_first = acTransList.take(1)
data_first

[('SB10001', 1000)]

In [ ]:
.takeSample(...)


In [50]:
data_sampled = acTransList.takeSample(False,1,666)
data_sampled

[('SB10002', 1200)]

.reduce(...)

In [51]:
rdd1.map(lambda x:x[1]).reduce(lambda x,y :x+y)
#recuce函数需要具有associative和communicative

12

In [54]:
data_reduce =sc.parallelize([1,2,.5,.1],1)#数据分成了一个partition

In [55]:
works = data_reduce.reduce(lambda x,y :x/y)
works

10.0

In [57]:
data_reduce_1 =sc.parallelize([1,2,.5,.1],3)#数据分成了三个partition

In [58]:
works = data_reduce_1.reduce(lambda x,y :x/y)
works

0.1

In [66]:
data_key =sc.parallelize([("a",1),("b",1),("c",10),("a",1),("b",5),("c",2),("d",7)])
data_key.reduceByKey(lambda x,y:x+y).collect()

[('b', 6), ('c', 12), ('a', 2), ('d', 7)]

.count()

In [61]:
data_reduce.count()

4

In [62]:
len(data_reduce.collect())
#错误的做法，不要这样做

4

In [64]:
data_key.countByKey().items()

dict_items([('a', 2), ('b', 2), ('c', 2), ('d', 1)])

.saveAsTextFile(...)

In [68]:
data_key.saveAsTextFile('datakey1.txt')

In [73]:
def parseInput(row):
    import re
    pattern = re.compile(r'\(\'([a-z])')\',([0-9])\)')
    row_split =pattern.split(row)
    return (row_split[1],int(row_split[2]))

data_key_reread =sc\
    .textFile('datakey1.txt')
    .map(parseInput)
data_key_reread.collect()

SyntaxError: unexpected character after line continuation character (<ipython-input-73-9c8339ebeac3>, line 3)

.foreach(...)#将方法应用于每一个element，但是不返回结果


In [74]:
def f(x):
    print(x)
data_key.foreach(f)#cli查看结果

In [75]:
data_key.collect()

[('a', 1), ('b', 1), ('c', 10), ('a', 1), ('b', 5), ('c', 2), ('d', 7)]